In [1]:
import gymnasium as gym
import numpy as np
from tqdm import tqdm
from gymnasium.wrappers import RecordVideo
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distributions
import torch.optim as optim
import matplotlib.pyplot as plt

env = gym.make(
    "LunarLander-v3",
    continuous=False,     
    gravity=-10.0,        
    enable_wind=False,   
    wind_power=15.0,      
    turbulence_power=1.0, 
    render_mode="rgb_array" 
)

In [ ]:
space_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Training parameters (following paper)
gamma = 0.99                # Discount factor
lr = 3e-4                   # Learning rate
lamb = 0.95                 # Generalised Advantage Estimation (GAE) lambda
epsilon = 0.2               # Clipping value
h = 0.01                    # Entropy coefficient
v = 0.5                     # Value loss coefficient
max_timesteps = 1e6         # Maximal number of iterations
N = 1                       # Number of agents collecting training data
T = 2048                    # Maximal trajectory length
K = 10                      # Number of epoches per update
minibatch_size = 64         # Size of a mini batch
number_minibatches = N * T / minibatch_size     # Number of mini batches
actor_losses = []
critic_losses = []

In [3]:
# The network to select an action
ActorNetwork = nn.Sequential(
    nn.Linear(space_dim, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 128),
    nn.LeakyReLU(),
    nn.Linear(128, action_dim)
)

# The network to get value of a state
CriticNetwork = nn.Sequential(
    nn.Linear(space_dim, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 1)
)

# Optimizer using Adam Gradient Descent
actor_optimizer = optim.Adam(ActorNetwork.parameters(), lr=lr)
critic_optimizer = optim.Adam(CriticNetwork.parameters(), lr=lr)

In [4]:
"""
GAE estimates the advantage of taking an action in a state
"""
def compute_GAE(next_value, rewards, values, dones):
    advantages = []
    GAE = 0
    for t in reversed(range(len(rewards))):
        delta = rewards[t] + gamma * next_value * (1 - dones[t]) - values[t]      # TD error
        GAE = delta + gamma * lamb * (1 - dones[t]) * GAE
        advantages.insert(0, GAE)
        next_value = values[t]
    return advantages

In [ ]:
state, _ = env.reset()      # Initialize state s_t
timesteps = 0

while timesteps < max_timesteps:
    states, actions, rewards, log_probs, values, dones = [], [], [], [], [], []

    for _ in range(T):              # Collect T timesteps for 1 rollout
        state_tensor = torch.FloatTensor(state)
        value = CriticNetwork(state_tensor)
        action_pred = ActorNetwork(state_tensor)                       # Select action a_t
        dist = distributions.Categorical(action_pred)
        action = dist.sample()
        log_prob = dist.log_prob(action)        # Old policy given action a_t

        next_state, reward, terminated, truncated, _ = env.step(action.item())      # Advance simulation one time step
        done = terminated or truncated

        # Collect training data
        states.append(state)         # Collect states
        actions.append(action)        # Collect actions from Actor Network
        rewards.append(reward)        # Collect rewards
        log_probs.append(log_prob)      # Collect lob_probs
        values.append(value)         # Collect values from Critic Network
        dones.append(done)          # Collect done (0 or 1)

        state = next_state      # Move to next state
        timesteps += 1          # Increase timesteps

        if done:
            state, _ = env.reset()      # If an episode is done, reset the state
    
    # Compute advantages
    next_state_tensor = torch.FloatTensor(state)            # Get the last state reached after last action
    next_value = CriticNetwork(next_state_tensor)           # Calculate the next value
    advantages = compute_GAE(next_value, rewards=rewards, values=values, dones=dones)       # Calculate advantage using GAE
    advantages = torch.FloatTensor(advantages)
    returns = advantages + torch.FloatTensor(values)        # Calculate V-target_t = A_t + V-w(s_t)

    # Optimize policy
    states = torch.FloatTensor(states)
    actions = torch.LongTensor(actions)
    old_log_probs = torch.FloatTensor(log_probs)
    returns = returns.detach()

    for k in range(K):
        for i in torch.randperm(len(states)).split(minibatch_size):        # Sample over batches with size 'minibatch_size'      
            batch_states = states[i]
            batch_actions = actions[i]
            batch_old_log_probs = old_log_probs[i]
            batch_returns = returns[i]
            batch_advantages = advantages[i]

            optimized_value = CriticNetwork(batch_states)
            optimized_action_preds = ActorNetwork(batch_states)
            optimized_dist = distributions.Categorical(optimized_action_preds)
            optimized_log_probs = optimized_dist.log_prob(batch_actions)
        
            ratio = (optimized_log_probs - batch_old_log_probs).exp()
            unclipped_objective = ratio * batch_advantages
            clipped_objective = torch.clamp(ratio, 1 - epsilon, 1 + epsilon) * batch_advantages

            L_clip = -torch.min(unclipped_objective, clipped_objective).mean()
            L_v = (optimized_value.squeeze() - batch_returns).pow(2).mean()
            H = optimized_dist.entropy().mean()

            # Loss = L_clip + v * L_v - h * H

            # Actor loss
            actor_loss = L_clip - h * H
            actor_optimizer.zero_grad()
            actor_loss.backward(retain_graph=True)  # Retain graph if value_loss uses same computation graph
            actor_optimizer.step()

            # Critic loss
            critic_optimizer.zero_grad()
            L_v.backward()
            critic_optimizer.step()
            
